In [ ]:
# Cell 1: Setup & Imports
%load_ext autoreload
%autoreload 2

import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import joblib

# Setup path
current_dir = Path.cwd()
root_dir = current_dir.parent
if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

from src import config, preprocessing, rf_classifier, evaluation, utils

# Khởi tạo Experiment
exp_paths = utils.setup_experiment_folder()
print(f"🚀 Experiment ID: {exp_paths['root'].name}")

# Cell 2: Load Both Datasets
print("🚀 Loading 2017 and 2018 datasets...")

# Nạp toàn bộ 2017 và 2018
X_17_raw, y_17_raw = preprocessing.load_single_dataset_year('2017', binary_mode=True)
X_18_raw, y_18_raw = preprocessing.load_single_dataset_year('2018', binary_mode=True)

print(f"✅ Loaded 2017: {X_17_raw.shape}")
print(f"✅ Loaded 2018: {X_18_raw.shape}")

# Cell 3: Feature Selection
# Lấy index của 25 đặc trưng mRMR đầu tiên từ config
mrmr_25_features = config.mRMR_FEATURES[:25]
mrmr_indices = [config.SELECTED_FEATURES.index(f) for f in mrmr_25_features]

# Chuyển đổi sang NumPy và chỉ lấy 25 đặc trưng
X_17 = X_17_raw.iloc[:, mrmr_indices].values
X_18 = X_18_raw.iloc[:, mrmr_indices].values

y_17 = y_17_raw.values if hasattr(y_17_raw, 'values') else y_17_raw
y_18 = y_18_raw.values if hasattr(y_18_raw, 'values') else y_18_raw

# Giải phóng dữ liệu thô
del X_17_raw, X_18_raw, y_17_raw, y_18_raw
import gc; gc.collect()

# Cell 4: Splitting & Scaling (Train 17, Test 18)
from sklearn.model_selection import train_test_split

# Kịch bản 2A: Train 2017 (100%) -> Test 2018 (20%)
X_train_2A = X_17
y_train_2A = y_17

# Lấy mẫu 20% của 2018 để làm tập test cho 2A (Stratified để giữ tỉ lệ nhãn)
_, X_test_2A, _, y_test_2A = train_test_split(
    X_18, y_18, test_size=0.2, random_state=config.SEED, stratify=y_18
)

print(f"📊 Scenario 2A: Train 17 ({X_train_2A.shape}) -> Test 18 ({X_test_2A.shape})")

# Cell 5: Scaling (Fit on Train ONLY)
scaler = preprocessing.get_scaler()

# Fit trên 2017 (tập train) và transform cả hai
X_train_2A_scaled = scaler.fit_transform(X_train_2A)
X_test_2A_scaled = scaler.transform(X_test_2A)

# Lưu scaler cho kịch bản này
joblib.dump(scaler, exp_paths["models"] / "scaler_cross_2A.joblib")
print("✅ Scaling completed. Scaler saved.")

# Cell 6: Training & Evaluation for Scenario 2A
# Huấn luyện mô hình RF
rf_model_2A = rf_classifier.train_rf(
    X_train_2A_scaled, 
    y_train_2A, 
    save_path=exp_paths["models"] / "rf_cross_2017_to_2018.joblib"
)

# Đánh giá
print("\n=== EVALUATION REPORT: Cross-Dataset (2017 -> 2018) ===")
evaluation.evaluate_model(
    model=rf_model_2A, 
    X_test=X_test_2A_scaled, 
    y_test=y_test_2A, 
    save_dir=exp_paths["figures"], 
    dataset_name="RF_Cross_2017_to_2018"
)

📂 New Experiment Created: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251221_052117
🚀 Experiment ID: exp_20251221_052117
🚀 Loading 2017 and 2018 datasets...
🔄 Loading dataset year 2017 (Binary=True)...
✅ Loaded 2017. Shape: (2830743, 65)
🔄 Loading dataset year 2018 (Binary=True)...
✅ Loaded 2018. Shape: (9625148, 65)
✅ Loaded 2017: (2830743, 65)
✅ Loaded 2018: (9625148, 65)
📊 Scenario 2A: Train 17 ((2830743, 20)) -> Test 18 ((1925030, 20))
✅ Scaling completed. Scaler saved.
[RandomForest] Training classifier...


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 100building tree 2 of 100
building tree 3 of 100

building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100building tree 31 of 100

building tree 32 of 100
building tree 33 of 100


[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   23.6s


building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75

[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:  1.3min finished


✅ RF Model saved to: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251221_052117\models\rf_cross_2017_to_2018.joblib

=== EVALUATION REPORT: Cross-Dataset (2017 -> 2018) ===

📊 Evaluating on Cross_2017_to_2018...


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    1.7s finished


   ✅ Accuracy: 0.8629
   ⭐ MCC:      0.6505
   📝 Report saved to: report_Cross_2017_to_2018.txt
   🖼️ Confusion Matrix saved to: cm_Cross_2017_to_2018.png


{'accuracy': 0.8629475904271622,
 'mcc': 0.6505013938289449,
 'report': {'Normal': {'precision': 0.8763462667485572,
   'recall': 0.9409459037955246,
   'f1-score': 0.907497917362048,
   'support': 1375383.0},
  'Attack': {'precision': 0.8188064542755226,
   'recall': 0.6677722247187741,
   'f1-score': 0.7356169105769269,
   'support': 549647.0},
  'accuracy': 0.8629475904271622,
  'macro avg': {'precision': 0.8475763605120399,
   'recall': 0.8043590642571493,
   'f1-score': 0.8215574139694874,
   'support': 1925030.0},
  'weighted avg': {'precision': 0.85991712782274,
   'recall': 0.8629475904271622,
   'f1-score': 0.8584213420689765,
   'support': 1925030.0}}}

In [1]:
############
# Cell 1: Setup & Imports for Reverse Cross-Dataset
%load_ext autoreload
%autoreload 2

import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import joblib

# Setup path
current_dir = Path.cwd()
root_dir = current_dir.parent
if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

from src import config, preprocessing, rf_classifier, evaluation, utils

# Khởi tạo Experiment mới cho kịch bản ngược
exp_paths = utils.setup_experiment_folder()
print(f"🚀 Experiment ID (Reverse Cross): {exp_paths['root'].name}")


# Cell 2: Load Both Datasets (2018 Train, 2017 Test)
print("🚀 Loading 2018 (Train source) and 2017 (Test target)...")

# Nạp dữ liệu
X_18_raw, y_18_raw = preprocessing.load_single_dataset_year('2018', binary_mode=True)
X_17_raw, y_17_raw = preprocessing.load_single_dataset_year('2017', binary_mode=True)

print(f"✅ Loaded 2018: {X_18_raw.shape}")
print(f"✅ Loaded 2017: {X_17_raw.shape}")

# Cell 3: Feature Selection
# Lấy index của 25 đặc trưng mRMR đầu tiên
mrmr_25_features = config.mRMR_FEATURES[:25]
mrmr_indices = [config.SELECTED_FEATURES.index(f) for f in mrmr_25_features]

# Sử dụng .iloc để truy xuất an toàn và chuyển sang NumPy array
X_18 = X_18_raw.iloc[:, mrmr_indices].values
X_17 = X_17_raw.iloc[:, mrmr_indices].values

y_18 = y_18_raw.values if hasattr(y_18_raw, 'values') else y_18_raw
y_17 = y_17_raw.values if hasattr(y_17_raw, 'values') else y_17_raw

# Giải phóng RAM ngay lập tức
del X_18_raw, X_17_raw, y_18_raw, y_17_raw
import gc; gc.collect()

# Cell 4: Splitting & Scaling (Train 18, Test 17)
from sklearn.model_selection import train_test_split

# Kịch bản 2B: Train 2018 (100%) -> Test 2017 (20%)
X_train_2B = X_18
y_train_2B = y_18

# Lấy mẫu 20% của 2017 để làm tập test
X_test_2B = X_17
y_test_2B = y_17

print(f"📊 Scenario 2B: Train 18 ({X_train_2B.shape}) -> Test 17 ({X_test_2B.shape})")

# Cell 5: Scaling (Fit on Train ONLY)

scaler = preprocessing.get_scaler()

# Fit trên tập huấn luyện (2018)
X_train_2B_scaled = scaler.fit_transform(X_train_2B)
# Transform tập kiểm thử (2017)
X_test_2B_scaled = scaler.transform(X_test_2B)

# Lưu scaler cho kịch bản 2B
joblib.dump(scaler, exp_paths["models"] / "scaler_cross_2B.joblib")
print("✅ Scaling completed. Scaler saved.")

# Cell 6: Training & Evaluation for Scenario 2B
# Huấn luyện mô hình RF trên dữ liệu 2018
rf_model_2B = rf_classifier.train_rf(
    X_train_2B_scaled, 
    y_train_2B, 
    save_path=exp_paths["models"] / "rf_cross_2018_to_2017.joblib"
)

# Đánh giá trên dữ liệu 2017
print("\n=== EVALUATION REPORT: Cross-Dataset (2018 -> 2017) ===")
evaluation.evaluate_model(
    model=rf_model_2B, 
    X_test=X_test_2B_scaled, 
    y_test=y_test_2B, 
    save_dir=exp_paths["figures"], 
    dataset_name="RF_Cross_2018_to_2017"
)

📂 New Experiment Created: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251221_053005
🚀 Experiment ID (Reverse Cross): exp_20251221_053005
🚀 Loading 2018 (Train source) and 2017 (Test target)...
🔄 Loading dataset year 2018 (Binary=True)...
✅ Loaded 2018. Shape: (9625148, 65)
🔄 Loading dataset year 2017 (Binary=True)...
✅ Loaded 2017. Shape: (2830743, 65)
✅ Loaded 2018: (9625148, 65)
✅ Loaded 2017: (2830743, 65)
📊 Scenario 2B: Train 18 ((9625148, 20)) -> Test 17 ((2830743, 20))
✅ Scaling completed. Scaler saved.
[RandomForest] Training classifier...


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 100building tree 2 of 100
building tree 3 of 100

building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100building tree 13 of 100

building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100building tree 21 of 100

building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100


[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:  2.3min


building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75

[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:  7.7min finished


✅ RF Model saved to: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251221_053005\models\rf_cross_2018_to_2017.joblib

=== EVALUATION REPORT: Cross-Dataset (2018 -> 2017) ===

📊 Evaluating on Cross_2018_to_2017...


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    3.3s finished


   ✅ Accuracy: 0.8315
   ⭐ MCC:      0.3384
   📝 Report saved to: report_Cross_2018_to_2017.txt
   🖼️ Confusion Matrix saved to: cm_Cross_2018_to_2017.png


{'accuracy': 0.8315445803451602,
 'mcc': 0.3383803244909627,
 'report': {'Normal': {'precision': 0.8312318202119228,
   'recall': 0.9915331373892096,
   'f1-score': 0.9043337059509413,
   'support': 2273097.0},
  'Attack': {'precision': 0.8386539686797894,
   'recall': 0.17939337859502266,
   'f1-score': 0.2955637953702746,
   'support': 557646.0},
  'accuracy': 0.8315445803451602,
  'macro avg': {'precision': 0.8349428944458561,
   'recall': 0.5854632579921161,
   'f1-score': 0.5999487506606079,
   'support': 2830743.0},
  'weighted avg': {'precision': 0.8326939562675492,
   'recall': 0.8315445803451602,
   'f1-score': 0.784408263918349,
   'support': 2830743.0}}}